# New sectioned configuration file help

This document explains how to write a configuration file with sections and subsections, allowing the user to provide their own functions for e.g., the mass-observable relation and its intrinsic scatter.

**Important Note:**
The document below is a minimalist description of the configuration file, and not all parameters or components are included. The full configuration file required by `halo.model` can be found in the `demo` folder within the pipeline directory tree.

## 1) Model

The configuration file starts with the model specification. This follows the same syntax as before:

```
model            halo.model
```

## 2) Model ingredients

### 2.1) HOD observables

The first section is the `observables` section:

```
[observables]
logmstar       10,11,12.5       10.4,11.5      log
```

The first column here is an arbitrary (and dummy) name for the observable in question; the second column are the bin limits used in the stacking (comma-separated), and the third column are comma-separated averages in each of the bins. In this case, we use two bins in the ranges $10 \leq \log_{10} m_\star \leq 11$ and $11 \leq \log_{10} m_\star \leq 12.5$, and the average log-stellar masses in those bins are $\langle \log_{10} m_\star \rangle=(10.4,11.5)$, respectively. Note that the average may be defined as whatever the user wishes and is not really used in the analysis, but we keep it here in case it is used somehow in the future.

The final column is optional, and for now its only possible value is `log`. This tells the pipeline that the observable is given in log-space. This is necessary information for properly applying the HOD (i.e., mass-observable relation and mass-observable scatter function, a.k.a. occupation probability). If the column is not present, it is understood that the observable is given in linear space. Any value different than `log` will raise an `AssertionError`.

Note that the HOD itself does not care what observable it receives so long as everything is passed consistently. It is possible, for instance, to define an observable $m_\star^{12} \equiv m_\star/10^{12}$. In the configuration file,

```
[observables]
mstar12        0.01,0.1,3.2       0.025,0.32
```

We will see what this implies for the rest of the configuration file below (note that $m_\star^{12}$ is not in log space, therefore we omit the fourth column).

### 2.2) Selection function

The second section concerns the selection function:

```
[selection]
selection_file.txt       z,logmstar,completeness      fixed_width
```

Columns here correspond to the file name, the columns to be used, and the `astropy.io.ascii` format to be used when reading the file. We recommend always using `fixed_width` with its default settings to avoid any confusion. A file generated with `ascii.write(*args, format='fixed_width')` is also easily human-readable:

```
|    z |      Msz | completeness |
| 0.10 | 2.00e+13 |     3.36e-01 |
| 0.15 | 2.00e+13 |     3.36e-01 |
| 0.20 | 2.00e+13 |     3.36e-01 |
| 0.30 | 2.00e+13 |     3.36e-01 |
| 0.40 | 2.00e+13 |     3.36e-01 |
| 0.50 | 2.00e+13 |     3.36e-01 |
| 0.60 | 2.00e+13 |     3.36e-01 |
| 0.80 | 2.00e+13 |     3.36e-01 |
| 1.00 | 2.00e+13 |     3.36e-01 |
```

Column names specified in the second column above must be in that order: (redshift,observable,completeness). They will be recorded in that order irrespective of the order in which they are present in the file.

This section is the only section that can be disabled altogether. We've chosen to allow this so the user is not required to produce a selection file if they do not have/want one. To ignore selection effects -- or, equivalently, to assume that the sample is 100% complete -- simply write

```
[selection]
None
```

### 2.3) Halo model ingredients

The third section is the `ingredients` section:

```
[ingredients]
centrals       True
pointmass      True
satellites     False
miscentring    False
```

Each of the ingredients' name is now fixed (as `ingredients` is a dictionary in the pipeline), and their values are either `True` (used) or `False` (not used). In the example above, we're asking the pipeline to evaluate a model that includes only central galaxies that are always located at the center of the gravitational potential, plus a point-mass component to account for stellar mass in the central galaxy.

## 3) Model parameters

From now on, things change. Like above, we now define model *sections*, which refer to each of the
components of the halo model. The order of sections must be followed for the model
to be set up properly, but each component may have a custom number of parameters. *This is the whole idea behind the new configuration file structure, and allows full flexibility in the model used without having to modify the backbone provided by `halo.model`.*:

```
[section1/subsection1]
name1         prior1     [values1 ...]
name2         prior2     [values2 ...]
...
[section1/subsection1/subsubsection1]
name1         prior1     [values1 ...]
name2         prior2     [values2 ...]
...
[section2/subsection1]
name1         prior1     [values1 ...]
name2         prior2     [values2 ...]
...
```

etc, etc. This will be enough to explain what's going on (empty lines and lines starting with `'#'` are ignored). The first column is the name that will be used in the MCMC output if the parameter is varied during the chain; the second specifies the prior function, and following columns specify parameters passed to the prior function. See the [priors](priors.ipynb) section for details.

### 3.1) Cosmological parameters

The first section including parameters that may be sampled in the MCMC is the section listing cosmological parameters:

```
[cosmo]
sigma_8         fixed     0.8159
H0              fixed     67.74
omegam          fixed     0.3089
omegab_h2       fixed     0.02230
omegav          fixed     0.6911
n               fixed     0.9667
z               array     0.188
```

They need not all be fixed, but in this example they are. At any rate, this will always be just a list of parameters, so no major changes compared to the previous version. *Note that the section name **must** be `cosmo`, and the list of cosmological parameters is fixed.*

Note however that we introduced a new variable type: `array`. This replaces the old `hm_params` (with an "s" at the end), and refers to values that should always be treated as arrays; the values thus defined are always fixed. Redshift is the prototypical `array` variable in `halo.model`.

### 3.2) HOD parameters

Now we move on to the HOD proper, and this is where the fun starts. The following are sections that can be modified seamlessly to produce a variety of halo model prescriptions, taking advantage of the backbone established by `halo.model` (i.e., the user need not write their own full-fledged model to do this):

```
[hod/centrals]
fc             uniform        0.1     5
bias           fixed          1
[hod/centrals/mor]
name           powerlaw
logM0         fixed          14.0
a              uniform        -1      1
b              student        1
[hod/centrals/scatter]
name           lognormal
sigma_c        jeffrey
[hod/centrals/miscentring]
name           fiducial
p_off          uniform        0       1
R_off          uniform        0       1.5
```

The whole idea behind this structure is that the HOD may be fully specified by the user, including for instance the complexity of the mass-observable scaling relation. Note that the HOD may also contain a model for satellites and potentially other ingredients, but a simple centrals-only model will serve our purpose here (but note that `halo.model` does require satellite sections to be defined; please refer to `demo/ggl_model_demo.txt` for a full working configuration file). While it is the order of the sections that is required by the halo model the `hod/` prefix to all HOD sections is required for the configuration file to be read properly. In any case, we recommend that the section names not be modified for consistency and ease of interpretation by other users.

In the example above we've only included mandatory parameters for each prior type, to keep it simple. Note also that we introduced new priors here compared to `v1.x` (and the `name` parameter). For more information see the [priors](priors.ipynb) section.

***Note:*** *The miscentring implementation has not yet been modified from `v1.x`, and therefore the `name` parameter is silent for now (but still must be defined and given a value). No matter the value given, miscentring will be modelled as in Viola et al. (2015). If anyone should require more flexibility please raise an issue and we will make this a more urgent update.*

## 4) Setup

There is an additional section, `setup`, which would include things like the $k$- or halo mass-binning schemes; e.g.,

```
[setup]
lnk_min          -13
lnk_max          13
kbins            1000
```

that is, essentially the `setup` section should include any parameter in the halo model that would *never* be a free parameter (not even a nuisance parameter); for instance, binning schemes or any precision-setting values.

## 5) Model output

In addition, the configuration file should include a section `output`, containing any outputs produced by the model in addition to the free parameters. These are given as a name and the data format to be used in the FITS file, in addition to the number of dimensions, if applicable. The typical FITS format would be `E`, corresponding to single-precision floating point. See the [astropy help](http://docs.astropy.org/en/stable/io/fits/usage/table.html#column-creation) for more details.

You will usually want to have each ESD component here at the very least. `halo.model` outputs the total ESD and the effective halo mass per bin. In our 2-bin example, we would write

```
[output]
esd            2,8E
Mavg           2,E
```

where the first line means to register two separate columns, each with elements corresponding to arrays of length 8 (the 8 R-measurements that make up the ESD profile); and the second line means to create two other columns each containing scalars corresponding to the effective halo masses (this is given by the output of `halo.model`, *not* by the name given in the first column above).

The first column corresponds to the names given to the columns in the output FITS file. When there is more than one "dimension" (there are 2 in this case), columns are labelled e.g., `esd1,esd2,...`.

There is one alternative to the example above:

```
[output]
esd       2,8E
Mavg        2E
```

which, consistent with the description above, will produce a column `Mavg` where each entry contains both masses, rather than making separate columns.

## 6) Sampler configuration

And finally the `sampler` section, which remains the same:

```
[sampler]
path_data            path/to/data
data                 shearcovariance_bin_*_A.txt     0,1,4
path_covariance      path/to/covariance
covariance           shearcovariance_matrix_A.txt    4,6
```

where `path_data` and `path_covariance` are optional. Note the (optional) use of a wildcard (`*`) in `data`: the pipeline will then select more than one file if available. Note that the file names must be such that, when sorted alpha-numerically, they are sorted in increasing observable binning. (This is properly taken care of by the KiDS-GGL ESD production pipeline).

The third column in `data` specifies which columns from the data should be used: R-binning column, ESD column, and multiplicative bias correction column. Similarly, the third column in `covariance` specifies the covariance column and the multiplicative bias correction column. The covariance file should follow the format produced by the ESD production part of this same pipeline. In both cases, the multiplicative bias correction column is *optional* (if the correction has already been applied). The numbers used above correspond to those required if the data come from the KiDS-GGL ESD production pipeline.

The `sampler` section then continues with a few more settings:

```
exclude              11,12              # bins excluded from the analysis (count from 0)
sampler_output       output/model.fits  # output filename (must be .fits)
sampler              emcee              # MCMC sampler (fixed)
nwalkers             100                # number of walkers used by emcee
nsteps               2000               # number of steps per walker
nburn                0                  # size of burn-in sample
thin                 1                  # thinning (every n-th sample will be saved, but values !=1 not fully tested)
threads              3                  # number of threads (i.e., cores)
sampler_type         ensemble           # emcee sampler type (fixed)
update               20000              # frequency with which the output file is written
```

where only `exclude` is optional.

# Running the improved KiDS-GGL pipeline

## I want to use it, please!

The file `demo/ggl_model_demo.txt` is ready to be used with `v2` of the KiDS-GGL pipeline. Simply download it and modify the values as needed.

## Coding up your own model

With all these sections and parameters, the pipeline can interpret any model passed to it, no matter the number of parameters or the ordering of the ingredients, provided that the configuration file is consistent with the model structure.

For instance, we might define a mass-observable relation with a power-law with mass and redshift,
$$
\log m_\star = A + B\log\left(\frac{M_\mathrm{h}}{M_0}\right) + C\left(\frac{1+z}{1+z_0}\right)\,,
$$
which coded up looks like:

In [7]:
def powerlaw_mz(M, z, A, B, C, logM0, z0):
    return 10**(A + B*(np.log10(M)-logM0) + C*((1+z)/(1+z0)))

Note that these custom functions must include the halo mass as the first argument; all other arguments must be
defined in the configuration file -- including the redshift, in this case. This would therefore require the central MOR section in the configuration file to look like

```
[hod/centrals/mor]
name           powerlaw_mz
z              fixed          0.188
A              uniform        10      16    12
B              uniform        0       5     1
C              uniform        -1      1     0
logM0          fixed          14
z0             fixed          0.1
```

For the time being, you must include your model in the file `kids_ggl_pipeline/halomodel/hod/relations.py`, and custom functions for the scatter about this relation must be included in the file `kids_ggl_pipeline/halomodel/hod/scatter.py`. This has the undesirable effect that this file, common to all users, might get clogged with trial-and-error attempts by various users, so please try to only push files with working models (and if you have a new working model, please push it!). We will implement the ability to input user-provided files in the future.

*The only condition is that the order of sections **must follow the order defined in the coded model**.*

## Future improvements

 * Choosing between mass-concentration relations should be trivial under this scheme. Should add a subsection `[hod/centrals/concentration]` and possibly one for satellites
 * Custom relations should be written in a user-supplied file rather than in the pipeline source code.
 * Adding a `module_path` optional entry to each section would easily allow custom files: the pipeline could simply add that path to `sys.path` and import from there.
     * There is the pickling problem however. Need to check if the above would allow for multi-thread runs.
 * Might want to add a `model` section, in case the above is implemented but more generally for any future changes